In [2]:
# import the user dataset, for which each observation is a user who was approved for a life insurance product
# some of the users purchased and some did not - this is the target variable

import pandas as pd
user = pd.read_csv(r'C:\Users\milan\Documents\GitHub\DSC-630-Project\Propensity.csv')
user.head()

C:\Users\milan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20,21,30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,application_id,bind_date,open_to_submit_min,submit_to_approve_min,channel_combined,business_line,open_date,bind,pre_approval_interactions,post_approval_pre_bind_interactions,...,travel_countries_count,med_conditions_count,bmi,age,income,annual_premium_estimated,income_comb,income_grp_v1,income_grp_v2,income_prem_ratio
0,cc3090db-2ed8-4fc5-8f66-f3ef8bfef12b,2021-08-01,5,2,Lemonade,protect,2021-07-24,1,0,0,...,NaN,1.0,26.58,57.53,360000.0,1112.04,360000.0,$100k+,over $50k,323.729362
1,9bffeb0e-bf7d-4544-a175-df737563b530,NaN,4,2,Lemonade,protect,2021-08-12,0,0,0,...,NaN,1.0,33.90,41.32,57000.0,NaN,57000.0,$50k-$100k,over $50k,NaN
2,38c5539f-aa55-406f-8356-91859e67502a,NaN,0,2,Lemonade,protect,2021-08-22,0,0,0,...,NaN,1.0,22.96,39.60,84000.0,NaN,84000.0,$50k-$100k,over $50k,NaN
3,ea0c0f26-b08a-4121-92b4-04b36ac16442,2021-08-10,4,2,Lemonade,protect,2021-08-10,1,0,0,...,NaN,1.0,34.17,41.93,245000.0,2541.96,245000.0,$100k+,over $50k,96.382319
4,0a40356d-aaa0-4aaa-9bbd-5f0f0534608c,NaN,9,1,Lemonade,protect,2021-12-04,0,0,0,...,NaN,1.0,37.12,42.82,78500.0,NaN,78500.0,$50k-$100k,over $50k,NaN


In [3]:
user.shape

(127691, 61)

In [4]:
# import the quote dataset, where each observation is an application id that includes the premiums that a user was shown 
# at quote and after being approved for the same product and face value, which is not always the same
# in fact, a previous analysis showed that more than half the time the premium the user is shown after 
# having their application approved is higher than the one they they were shown when they got a quote

# a key question to be answered in this model is the importance of this feature

quote = pd.read_csv(r'C:\Users\milan\Documents\GitHub\DSC-630-Project\quote_to_approved.csv')
quote.head()

,created_at,business_line,quote_id,application_id,face_amount,product,quoted_risk_class,approved_risk_class,quoted_premium,approved_premium,premium_difference,premium_pct_diff
0,2019-10-27,direct,47386ebc-4405-40ca-b2c1-79b24a537e97,844a2118-1c9e-4f50-86ed-970937f8ecfc,200000,BT2002,select_tb,select_nt,5.0,33.33,-28.33,-5.666
1,2022-04-11,direct,ee741da6-788e-41cb-8b23-ab23bf07fecc,e3f51c27-eba2-4f36-86d2-f204baf66455,50000,BT1004,elite_nt,elite_nt,7.5,7.50,0.00,0.000
2,2021-10-26,direct,f05908fa-25df-4219-89fb-b30f2bb517a3,8cf1757c-f644-4082-a0bb-8b3ecb50d79b,50000,BT1004,elite_nt,elite_nt,7.5,7.50,0.00,0.000
3,2021-04-28,direct,4e8cf39c-5334-451f-8770-d3de35d1b4ea,42c460b7-3d17-4c27-8f11-535b5da44d2f,50000,BT1004,elite_nt,super_elite_nt,7.5,7.50,0.00,0.000
4,2021-04-19,advisor,e573f1be-9ed2-4ff9-8597-20859417cad0,7145a861-c133-4330-918b-80d4199078ca,50000,BT1004,elite_nt,super_elite_nt,7.5,7.50,0.00,0.000


In [5]:
quote.shape

(101416, 12)

In [6]:
# the user dataframe has ~127k observations
# the quote dataframe has ~101k observations
# about 45k observationa are in both dataframes

df = pd.merge(user, quote, on = ['application_id'])
df.shape

(44844, 72)

In [7]:
# check the bind field as it is our target
df['bind'].unique()

array([0, 1], dtype=int64)

In [8]:
df['bind'].isnull().sum()

0

In [28]:
df.groupby('bind')['application_id'].count().reset_index()

,bind,application_id
0,0,25607
1,1,19237


Possible values for bind (which means that the user purchased a policy) are 0 and 1, where 0 means that they did not bind.  There are no null values, and the dataset, while not perfectly balanced, is not unbalanced, with about 56% of users not binding and 44% binding.

In [10]:
# list the columns and create a data dictionary
df.columns

Index(['application_id', 'bind_date', 'open_to_submit_min',
       'submit_to_approve_min', 'channel_combined', 'business_line_x',
       'open_date', 'bind', 'pre_approval_interactions',
       'post_approval_pre_bind_interactions', 'post_bind_interactions',
       'total_interactions', 'anonid_uniq', 'sessions_uniq', 'event_total',
       'session_pageload_total', 'url_uniq', 'total_seconds', 'gclid_count',
       'customer_portal_loads', 'device', 'device_name',
       'avg_seconds_per_session', 'avg_events_per_session', 'event_len',
       'utm_source_len', 'utm_medium_len', 'utm_campaign_len', 'utm_term_len',
       'referrer_host_len', 'record_created_at', 'gender', 'tb',
       'mvr_order_status', 'current_ins', 'rx_result', 'replacement_ins',
       'legal_resident', 'employment_status', 'postal_code_address',
       'income_payment_period', 'designate_secondary_contact',
       'alcohol_weekly', 'disability_pmts', 'term_length_years',
       'household_income', 'lrc_score', 'f

### Data Dictionary

#### From user
- application_id: categorical, unique identifier of the application
- bind_date: date, the date of when the applicant bound a policy (needs to be removed from model to avoid leakage)
- open_to_submit_min: integer, number of minutes to complete and submit the application
- submit_to_approve_min: integer, number of minutes for the application to get approved after it is submitted
- channel_combined: categorical, marketing channel(s) responsible for the application start ?  or the quote?  or something else?
- business_line: categorical, direct, protect, core, advisors
- open_date: date, the date the application was started
- bind: binary, indicating if the applicant bound a policy (target variable)
- anonid_uniq: number of anonymous id's from segment
- sessions_uniq: number of unique sessions
- event_total: number of events
- session_pageload_total: number of session page loads
- url_uniq: number of unique url's accessed by the user
- total_seconds:  how long on the web site
- gclid_count: ?
- customer_portal_loads: ?
- device: device type, i.e., phone, tablet, desktop
- device_name: device type + OS
- avg_seconds_per_session: average time per session in second
- avg_events_per_session: average number of events per session
- event_len: average number of seconds per event
- gender: gender of applicant
- tb: binary, tobacco user or not
- current_ins: binary, does the applicant currently have life insurance
- rx_result: (remove)
- replacement_ins: binary, does the applicant currently have replacement insurance (what is replacement insurance?)
- legal_resident: resident or visa
- employment_status:  'fulltime', 'parttime', 'fulltime_student', 'retired','stay_at_home', 'parttime_student', 'unemployed'
- postal_code_address: zip code of home address
- income_payment_period: how frequently the applicant gets paid
- designate_secondary_contact:  no non-null values, so this can be removed
- alcohol_weekly: number of drinks per week as reported by the applicant
- disability_pmts: binary, does (or has?) the applicant receive(d) diability payments (history)
- term_length_years: length of term (this will likely be removed, but compare to term length from quote)
- household_income: reported household income (verify)
- lrc_score: brought in from vendor risk score from lexus nexes, mortality risk score.  This generally highly correlated with being approved
- face_value: face value of the policy (this will likely be removed, but compare to face value from quote)
- previous_declined: binary, has the applicant been previously declined for life insurance
- travel_countries_count: number of countries the applicant has visited
- med_conditions_count: number of medical conditions the applicant has history
- bmi: body mass index of applicant
- age: age of applicant
- income: income of applicant
- annual_premium_estimated: the annual premium shown to the applicant after approval (this will likely be removed, but compare to premium from quote)
- income_comb: annual income
- income_prem_ratio: rato of income to premium (annual)
    - Note: add income to face amount ratio as another feature
    
#### From quote
- created_at: when the application was started
- business_line_y: business line, join on this 
- quote_id: quote id associated with the approved app
- face_amount: face amount of the approved policy
- product: procuct code, including term length
- quoted_risk_class: the risk class that the applicant was put it at quote
- approved_risk_class: the risk class the applicant was put it after approval
- quoted_premium: the monthly premium shown at quote
- approved_premium: the monthly premium shown after approval
- premium_difference: the difference between the two premiums (quote - approved)
- premium_pct_diff: the percent difference between the two premiums (quote - approved)/quote

#### After discussing with the data science team, the following fields can be removed
- the following two field should be removed to avoid leakage
    - post_bind_interactions:
    - total_interactions: sum of the previous three fields (inlcuding post bind interactions)
- utm_source_len: not sure how the length of the various parts of the utm is relevant to predicting bind
- utm_medium_len
- utm_campaign_len
- utm_term_len
- referrer_host_len
- record_created_at: when the data was pulled
- mvr_points
- mvr_order_status
- zip_d1
- income_grp_v1: binned income group: under 50k, 50k-100k, 100k+ 
- income_grp_v2: binned income group; under 50k, 50k+

In [11]:
df = df[['application_id', 'open_to_submit_min',
       'submit_to_approve_min', 'channel_combined', 'business_line_x',
       'open_date', 'bind', 'pre_approval_interactions',
       'post_approval_pre_bind_interactions', 'anonid_uniq', 'sessions_uniq', 'event_total',
       'session_pageload_total', 'url_uniq', 'total_seconds', 'gclid_count',
       'customer_portal_loads', 'device', 'device_name',
       'avg_seconds_per_session', 'avg_events_per_session', 'event_len', 'gender', 'tb', 'current_ins', 'rx_result', 'replacement_ins',
       'legal_resident', 'employment_status', 'postal_code_address',
       'income_payment_period',
       'alcohol_weekly', 'disability_pmts', 'term_length_years',
       'household_income', 'lrc_score', 'face_value',
       'previous_declined', 'travel_countries_count',
       'med_conditions_count', 'bmi', 'age', 'income',
       'annual_premium_estimated', 'income_comb', 'income_prem_ratio', 'created_at', 'business_line_y',
       'quote_id', 'face_amount', 'product', 'quoted_risk_class',
       'approved_risk_class', 'quoted_premium', 'approved_premium',
       'premium_difference', 'premium_pct_diff']]

In [12]:
df['income_comb'].isnull().sum()

6369

In [13]:
# check how income_prem_ratio is created (hypothesis: annual income divided by annual premium)
# to check this hypothesis, I will create a column for annual income divided by annual premium and compare it to
# the values in the income_prem_ratio field
test = df[[ 'income_comb', 'annual_premium_estimated', 'income_prem_ratio']]
test['income_ratio'] = test.apply(lambda x: x.income_comb/x.annual_premium_estimated, axis=1)

C:\Users\milan\AppData\Local\Temp/ipykernel_20092/3119235808.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['income_ratio'] = test.apply(lambda x: x.income_comb/x.annual_premium_estimated, axis=1)


In [14]:
test

,income_comb,annual_premium_estimated,income_prem_ratio,income_ratio
0,182000.0,NaN,NaN,NaN
1,52650.0,362.04,145.425920,145.425920
2,38584.0,302.04,127.744670,127.744670
3,55200.0,NaN,NaN,NaN
4,78864.0,NaN,NaN,NaN
...,...,...,...,...
44839,85000.0,NaN,NaN,NaN
44840,85000.0,1412.04,60.196595,60.196595
44841,85000.0,649.56,130.857811,130.857811
44842,85000.0,1368.00,62.134503,62.134503


In [15]:
#  drop NaN and check the remaining values
test = test.dropna(subset = ['annual_premium_estimated']).reset_index()
test.shape

(19237, 5)

In [16]:
test[test['income_prem_ratio'] != test['income_ratio']]
# this is supposed to show observations where these fields are different, but they look the same

,index,income_comb,annual_premium_estimated,income_prem_ratio,income_ratio
5,12,160000.0,1362.00,117.474302,117.474302
18,38,37960.0,402.00,94.427861,94.427861
21,44,50400.0,942.00,53.503185,53.503185
29,65,1768000.0,2613.00,676.616915,676.616915
33,76,50388.0,435.00,115.834483,115.834483
...,...,...,...,...,...
19220,44818,85000.0,852.00,99.765258,99.765258
19223,44822,85000.0,932.04,91.197803,91.197803
19228,44827,85000.0,279.48,304.136253,304.136253
19232,44833,85000.0,423.96,200.490612,200.490612


In [17]:
# find the difference between the fields and sum it to see how different they are
test['diff'] = test.apply(lambda x: x.income_prem_ratio - x.income_ratio, axis =1)
test['diff'].sum()

-2.3276852559143968e-11

The difference is essentially 0, so we know how income_prem_ratio is calculated

In [19]:
# check to see how premium, face value, and term length compare between the two datasets that were combined
# first create a new dataframe with the fields that I want to compare

compare = df[['annual_premium_estimated', 'approved_premium', 'face_value', 'face_amount', 'term_length_years', 'product']]
# the approved premium is a monthly premium, so we multiply by 12 to convert it to annual
compare['annual_approved_premium'] = compare.apply(lambda x: x.approved_premium*12, axis = 1)

C:\Users\milan\AppData\Local\Temp/ipykernel_20092/585787592.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare['annual_approved_premium'] = compare.apply(lambda x: x.approved_premium*12, axis = 1)


In [20]:
# extract the term length from the product
# for example , if the product is BT2004, the term length is 20, so we need to extract the middle two characters
# and convert it to an integer to compare with term length
compare['approved_term'] = compare['product'].str[:4].str[-2:].astype(int)

C:\Users\milan\AppData\Local\Temp/ipykernel_20092/1808066344.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare['approved_term'] = compare['product'].str[:4].str[-2:].astype(int)


In [21]:
# examine the number of null values for each of these fields
compare = compare[['annual_premium_estimated', 'annual_approved_premium', 'face_value',
       'face_amount', 'term_length_years', 'approved_term']]
compare.isnull().sum()

annual_premium_estimated    25607
annual_approved_premium         0
face_value                  15285
face_amount                     0
term_length_years               0
approved_term                   0
dtype: int64

We see that the annual premium estimated is missing more than 25k observations, and face value is missing over 15k.  We will still check to see how the remaining values compare, but will use annual_approved_premium and face amount for our model.

In [22]:
# create a difference field for each of these fields
# create a percent difference field for premium to get a sense for how far off these values are
# since face value and term length are really categorical, a percent difference doesn't make sense
compare['prem_diff'] = compare.apply(lambda x: abs(x.annual_premium_estimated - x.annual_approved_premium)/x.annual_approved_premium*100, axis=1)
compare['face_diff'] = compare.apply(lambda x: abs(x.face_value - x.face_amount)/x.face_amount*100, axis=1)
compare['term_diff'] = compare.apply(lambda x: abs(x.term_length_years - x.approved_term), axis=1)
compare['prem_pct_diff'] = compare.apply(lambda x: abs(x.annual_premium_estimated - x.annual_approved_premium)/x.annual_approved_premium*100, axis=1)
compare.head(100)

,annual_premium_estimated,annual_approved_premium,face_value,face_amount,term_length_years,approved_term,prem_diff,face_diff,term_diff,prem_pct_diff,face_pct_diff
0,NaN,4832.04,NaN,1000000,20,20,NaN,NaN,0.0,NaN,NaN
1,362.04,477.96,500000.0,700000,15,15,24.253076,28.571429,0.0,24.253076,28.571429
2,302.04,302.04,500000.0,500000,20,20,0.000000,0.000000,0.0,0.000000,0.000000
3,NaN,1017.00,NaN,750000,30,30,NaN,NaN,0.0,NaN,NaN
4,NaN,989.04,700000.0,700000,30,30,NaN,0.000000,0.0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95,132.96,132.96,50000.0,50000,10,10,0.000000,0.000000,0.0,0.000000,0.000000
96,1247.04,1247.04,500000.0,500000,10,10,0.000000,0.000000,0.0,0.000000,0.000000
97,NaN,591.00,50000.0,50000,20,20,NaN,0.000000,0.0,NaN,0.000000
98,NaN,992.04,500000.0,500000,30,30,NaN,0.000000,0.0,NaN,0.000000


In [23]:
# 71% of observations differ for premium
compare[(compare['prem_diff']!=0)].shape[0]/compare.shape[0]

0.7082775845152083

In [27]:
# the mean absolute percent difference in premium is about 10%
compare['prem_pct_diff'].mean()

10.218040045733458

In [24]:
# About 50% of observations differ for face value
compare[(compare['face_diff']!=0)].shape[0]/compare.shape[0]

0.5035233253055035

In [25]:
# term length agrees for every observation
compare[(compare['term_diff']!=0)].shape[0]/compare.shape[0]

0.0

Due both to the large number of missing values, and more importantly discussions with the data science team about these fields, we will use the fields that come from the 

In [29]:
df.columns

Index(['application_id', 'open_to_submit_min', 'submit_to_approve_min',
       'channel_combined', 'business_line_x', 'open_date', 'bind',
       'pre_approval_interactions', 'post_approval_pre_bind_interactions',
       'anonid_uniq', 'sessions_uniq', 'event_total', 'session_pageload_total',
       'url_uniq', 'total_seconds', 'gclid_count', 'customer_portal_loads',
       'device', 'device_name', 'avg_seconds_per_session',
       'avg_events_per_session', 'event_len', 'gender', 'tb', 'current_ins',
       'rx_result', 'replacement_ins', 'legal_resident', 'employment_status',
       'postal_code_address', 'income_payment_period', 'alcohol_weekly',
       'disability_pmts', 'term_length_years', 'household_income', 'lrc_score',
       'face_value', 'previous_declined', 'travel_countries_count',
       'med_conditions_count', 'bmi', 'age', 'income',
       'annual_premium_estimated', 'income_comb', 'income_prem_ratio',
       'created_at', 'business_line_y', 'quote_id', 'face_amount', 'p

In [30]:
df = df[['application_id', 'open_to_submit_min', 'submit_to_approve_min',
       'channel_combined', 'business_line_x', 'open_date', 'bind',
       'pre_approval_interactions', 'post_approval_pre_bind_interactions',
       'anonid_uniq', 'sessions_uniq', 'event_total', 'session_pageload_total',
       'url_uniq', 'total_seconds', 'gclid_count', 'customer_portal_loads',
       'device', 'device_name', 'avg_seconds_per_session',
       'avg_events_per_session', 'event_len', 'gender', 'tb', 'current_ins',
       'rx_result', 'replacement_ins', 'legal_resident', 'employment_status',
       'postal_code_address', 'income_payment_period', 'alcohol_weekly',
       'disability_pmts', 'household_income', 'lrc_score',
       'previous_declined', 'travel_countries_count',
       'med_conditions_count', 'bmi', 'age', 'income',
        'income_comb',
       'created_at', 'business_line_y', 'quote_id', 'face_amount', 'product',
       'quoted_risk_class', 'approved_risk_class', 'quoted_premium',
       'approved_premium', 'premium_difference', 'premium_pct_diff']]

In [31]:
# about 6k missing values for income
df['income_comb'].isna().sum()

6369

In [32]:
# add annual approved premium, face amount to premium ratio, and income to premium ratio
df['annual_approved_premium'] = df.apply(lambda x: x.approved_premium*12, axis = 1)
df['face_prem_ratio'] = df.apply(lambda x: x.face_amount/x.annual_approved_premium, axis = 1)
df['income_prem_ratio'] = df.apply(lambda x: x.income_comb/x.annual_approved_premium, axis = 1)

In [35]:
# about 60% of users saw an increase in premium from quote to approved (premium difference = quote - approved)
df[df['premium_difference'] < 0].shape[0]/df.shape[0]

0.6004593702613504

In [36]:
# the average premium change was an increase of about $30
df['premium_difference'].mean()

-30.194834537507667

In [37]:
# the average premium percent change was 56%
df['premium_pct_diff'].mean()

-0.5604121070288857